In [1]:

import json
import requests
image='golang'
registry_url='https://registry-1.docker.io'
auth_url='https://auth.docker.io'
svc_url='registry.docker.io'

def auth_token():
    url = f"{auth_url}/token?service={svc_url}&scope=repository:library/{image}:pull"
    r = requests.get(url)
    return r.json()['token']



ModuleNotFoundError: No module named 'requests'